In [65]:
# importamos las librerías con las que vamos a trabajar

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd

In [66]:
try:
    conexionpeliculas=psycopg2.connect(
    database="film",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432")
except OperationalError as e:
    if e.pgcode ==errorcodes.INVALID_PASSWORD:
        print("La constraseña es incorrecta")
    elif e.pgcode ==errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")

In [67]:
# Primero, deshacer la transacción fallida
conexionpeliculas.rollback()

cursorpeliculas = conexionpeliculas.cursor()
cursorpeliculas

<cursor object at 0x000001B6F6D76420; closed: 0>

In [68]:
# Tabla peliculas

# Tipo       object
# Titulo     object
# Año         int64
# Mes         int64
# Id         object
# generos    object

querydecreacion1 = """
    CREATE TABLE IF NOT EXISTS peliculas (
    tipo VARCHAR,
    titulo VARCHAR,
    año INT,
    mes INT,
    id_pelicula VARCHAR PRIMARY KEY,
    genero VARCHAR
);
"""

# Ejecutar la consulta y confirmar los cambios
cursorpeliculas.execute(querydecreacion1)
conexionpeliculas.commit()

# Tabla detalles

# titulo          object
# cali_imdb      float64
# direc_imbd      object
# guion_imbd      object
# argum_imbd      object
# durac_imbd      object
# id_pelicula     object

querydecreacion2 = """
    CREATE TABLE IF NOT EXISTS detalles_pelicula(
    titulo VARCHAR,
    cali_imdb DECIMAL,
    direc_imbd VARCHAR,
    guion_imbd VARCHAR,
    argum_imbd VARCHAR,
    durac_imbd VARCHAR,
    id_pelicula VARCHAR PRIMARY KEY,
    FOREIGN KEY (id_pelicula) REFERENCES peliculas(id_pelicula)
    on delete restrict
    on update cascade
);
"""

# Ejecutar la consulta y confirmar los cambios
cursorpeliculas.execute(querydecreacion2)
conexionpeliculas.commit()

# Tabla detalles actores

# actr            object
# año_nac          int64
# conocido_por    object
# que_hace        object
# premios         object

querydecreacion4 = """
    CREATE TABLE IF NOT EXISTS detalles_actores (
    nombre VARCHAR PRIMARY KEY,
    año INT,
    conocido VARCHAR,
    rol VARCHAR,
    premios VARCHAR,
);
"""

# Ejecutar la consulta y confirmar los cambios
cursorpeliculas.execute(querydecreacion4)
conexionpeliculas.commit()

# Tabla actores

# Movie ID        object
# Actriz_Actor    object

querydecreacion3 = """
    CREATE TABLE IF NOT EXISTS actores (
    id_pelicula VARCHAR,
    nombre VARCHAR,
    FOREIGN KEY (id_pelicula) REFERENCES peliculas(id_pelicula)
    FOREIGN KEY (nombre) REFERENCES detalles_actores (nombre)
    on delete restrict
    on update cascade
);
"""

# Ejecutar la consulta y confirmar los cambios
cursorpeliculas.execute(querydecreacion3)
conexionpeliculas.commit()



In [69]:
df_peliculas=pd.read_csv("../data/extra/peliculas.csv")
df_peliculas.sample(5)

,Tipo,Titulo,Año,Mes,Id,generos
1345,Short,Powrót,2013,6,tt2797786,Mystery
1226,Short,Obscurity,2018,3,tt13369410,Thriller
54,Short,Even If It's Wrong,2004,4,tt0419736,Action
352,Movie,Jump Tomorrow,2001,11,tt0273300,Comedy
1,Movie,Velai Kidaichiruchu,1990,8,tt0318803,Action


In [70]:
df_peliculas.dtypes

Tipo       object
Titulo     object
Año         int64
Mes         int64
Id         object
generos    object
dtype: object

In [71]:
lista_peliculas = [tuple(row) for row in df_peliculas.values]

In [72]:
df_detalles_peliculas=pd.read_csv("../data/extra/detalles_peliculas.csv")
df_detalles_peliculas.sample(5)

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
1320,Where's Dookie Cannata?,NaN,Desconocido,Desconocido,Desconocido,15min,tt1340139
763,I Think You Should Flee,NaN,Roger Brian Williams,Tracy Ann Lisa,After an unknown supernatural creature killed ...,Desconocido,tt4472982
469,La mujer del Presidente,NaN,Carlos del Castillo,Carmen Mirabal,Desconocido,Desconocido,tt12572368
715,Svarta Madam,7.4,Desconocido,Desconocido,Desconocido,24m,tt5868058
1265,Sam,NaN,Ludovica Musumeci,Ludovica Musumeci,Desconocido,5min,tt11761282


In [73]:
df_detalles_peliculas.dtypes

titulo          object
cali_imdb      float64
direc_imbd      object
guion_imbd      object
argum_imbd      object
durac_imbd      object
id_pelicula     object
dtype: object

In [74]:
lista_detalles_peliculas = [tuple(row) for row in df_detalles_peliculas.values]

In [75]:
df_actores=pd.read_csv("../data/extra/pelicula_artista.csv")
df_actores.sample(5)

,Movie ID,Actriz_Actor
443,tt0115826,Louis Gossett Jr.
5632,tt6220752,Aaron Dalla Villa
2647,tt2795608,Abe Mitchell
6371,tt10687070,G.M. Sundar
1620,tt11123316,Diego Gurpegui


In [76]:
df_actores["Actriz_Actor"].value_counts()

Actriz_Actor
Sin datos         4094
Shakti Kapoor        8
Anupam Kher          8
Gulshan Grover       6
Mohnish Behl         6
                  ... 
Keep Chambers        1
Jarod Johnson        1
Joe D'Aguanno        1
Darren Baxter        1
Fanta Berete         1
Name: count, Length: 9909, dtype: int64

In [77]:
df_actores.dtypes

Movie ID        object
Actriz_Actor    object
dtype: object

In [78]:
df_actores.shape

(14529, 2)

In [79]:
df_actores.drop_duplicates(inplace=True)

In [83]:
lista_actores = [tuple(row) for row in df_actores.values]

In [84]:
df_detalles_actores=pd.read_csv("../data/extra/detalles_artistas.csv")
df_detalles_actores.sample(5)

,actr,año_nac,conocido_por,que_hace,premios
6198,Mathilda May,1965,"'Lifeforce: Fuerza vital', 'The Jackal Chacal'...",Actor,5 wins & 1 nomination
6061,Mariola Fuentes,1970,"'Los abrazos rotos', 'Hable con ella', 'Dos ti...",Actor,no tiene premios
117,Adrian Titieni,1963,"'Los exámenes', 'Ilegitim', 'Puzzle', 'Din dra...",Actor,1 nomination
5331,Krishnakant Singh Bundela,1981,"'Laal Singh Chaddha', 'Marne Ke Dar Se Mere Di...",Actor,no tiene premios
4686,Jonathan Higgins,9999,"'L'heroi de Berlín', 'Los elegidos', 'La casa ...",Actor,no tiene premios


In [85]:
df_detalles_actores.dtypes

actr            object
año_nac          int64
conocido_por    object
que_hace        object
premios         object
dtype: object

In [ ]:
df_detalles_actores.drop_duplicates(inplace=True)

In [97]:
df_detalles_actores.shape

(9907, 5)

In [86]:
df_detalles_actores.nunique()

actr            9907
año_nac          115
conocido_por    9301
que_hace        1076
premios          872
dtype: int64

In [87]:
lista_detalles_actores = [tuple(row) for row in df_detalles_actores.values]

In [88]:
# Primero, deshacer la transacción fallida
conexionpeliculas.rollback()

cursorpeliculas = conexionpeliculas.cursor()
cursorpeliculas

<cursor object at 0x000001B6F6000120; closed: 0>

In [89]:
queryinsercion = "INSERT INTO peliculas (tipo, titulo, año, mes, id_pelicula, genero) VALUES (%s,%s,%s,%s,%s,%s);"
listavaloresinsercion=lista_peliculas
cursorpeliculas.executemany(queryinsercion,listavaloresinsercion)
conexionpeliculas.commit()

UniqueViolation: llave duplicada viola restricción de unicidad «peliculas_pkey»
DETAIL:  Ya existe la llave (id_pelicula)=(tt0320656).


In [ ]:
queryinsercion = "INSERT INTO detalles_pelicula (titulo, cali_imdb, direc_imbd, guion_imbd, argum_imbd, durac_imbd, id_pelicula) VALUES (%s,%s,%s,%s,%s,%s,%s);"
listavaloresinsercion=lista_detalles_peliculas
cursorpeliculas.executemany(queryinsercion,listavaloresinsercion)
conexionpeliculas.commit()

In [ ]:
queryinsercion = "INSERT INTO actores (id_pelicula, nombre) VALUES (%s,%s);"
listavaloresinsercion=lista_actores
cursorpeliculas.executemany(queryinsercion,listavaloresinsercion)
conexionpeliculas.commit()

UniqueViolation: llave duplicada viola restricción de unicidad «actores_pkey»
DETAIL:  Ya existe la llave (nombre)=(Kirsten Storms).


In [ ]:
queryinsercion = "INSERT INTO detalles_actores (nombre, año, conocido, rol, premios) VALUES (%s,%s,%s,%s,%s);"
listavaloresinsercion=lista_detalles_actores
cursorpeliculas.executemany(queryinsercion,listavaloresinsercion)
conexionpeliculas.commit()